In [1]:
import requests

In [2]:
hist_cov = "cmip6_daily_pr_historical"
proj_cov = "cmip6_daily_pr_projected"

hist_scenario = 0
proj_scenario = 3

lat = 64
lon = -147
model = 13  # ensemble

In [3]:
hist_get_cov_request_url = f"https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID={hist_cov}&SUBSET=model({model})&SUBSET=scenario({hist_scenario})&SUBSET=lat({lat})&SUBSET=lon({lon})&FORMAT=application/json"
proj_get_cov_request_url = f"https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID={proj_cov}&SUBSET=model({model})&SUBSET=scenario({proj_scenario})&SUBSET=lat({lat})&SUBSET=lon({lon})&FORMAT=application/json"

In [4]:
with requests.get(hist_get_cov_request_url) as r:
    print(hist_get_cov_request_url)
    if r.status_code != 200:
        print("Error:", r.status_code)
    hist_result = r.json()

with requests.get(proj_get_cov_request_url) as r:
    print(proj_get_cov_request_url)
    if r.status_code != 200:
        print("Error:", r.status_code)
    proj_result = r.json()

https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=cmip6_daily_pr_historical&SUBSET=model(13)&SUBSET=scenario(0)&SUBSET=lat(64)&SUBSET=lon(-147)&FORMAT=application/json
https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=cmip6_daily_pr_projected&SUBSET=model(13)&SUBSET=scenario(3)&SUBSET=lat(64)&SUBSET=lon(-147)&FORMAT=application/json


In [5]:
# all historical days 1950 - 2014
# 65 years * 365 = 23725
len(hist_result)

23725

In [6]:
# all projected days 2015 - 2100
# 86 years * 365 = 31390
len(proj_result)

31390

In [7]:
# split the result into years and store data in a dict
years = {}
for i in range(65):
    year = 1950 + i
    years[year] = hist_result[i * 365 : (i + 1) * 365]
for i in range(86):
    year = 2015 + i
    years[year] = proj_result[i * 365 : (i + 1) * 365]

In [8]:
# make a function to take a list of 365 data points (1 year of precip data),
# and compute indicator rx_nday: Maximum n–day precipitation (mm)
def compute_rx_nday(precip_data, n):
    max_nday = 0
    for i in range(len(precip_data) - (n - 1)):
        n_day_sum = sum(precip_data[i : i + n])
        if n_day_sum > max_nday:
            max_nday = n_day_sum
    return max_nday

In [9]:
# for each year in the dict, calculate r10mm and store in a new dict
rxnday_results = {}

n = 7

for year, data in years.items():
    rxnday = compute_rx_nday(data, n=n)
    rxnday_results[year] = rxnday

In [10]:
rxnday_results

{1950: 30.499999999999996,
 1951: 34.5,
 1952: 29.500000000000004,
 1953: 28.7,
 1954: 29.7,
 1955: 32.4,
 1956: 29.499999999999996,
 1957: 31.399999999999995,
 1958: 36.900000000000006,
 1959: 34.1,
 1960: 28.000000000000004,
 1961: 28.5,
 1962: 29.900000000000002,
 1963: 32.1,
 1964: 32.8,
 1965: 32.1,
 1966: 30.900000000000006,
 1967: 30.5,
 1968: 36.2,
 1969: 38.7,
 1970: 28.999999999999996,
 1971: 29.0,
 1972: 32.9,
 1973: 32.5,
 1974: 29.9,
 1975: 34.9,
 1976: 30.7,
 1977: 29.7,
 1978: 28.7,
 1979: 29.6,
 1980: 32.2,
 1981: 28.4,
 1982: 32.4,
 1983: 33.1,
 1984: 31.6,
 1985: 32.7,
 1986: 35.199999999999996,
 1987: 29.6,
 1988: 31.000000000000004,
 1989: 33.1,
 1990: 30.0,
 1991: 30.4,
 1992: 30.9,
 1993: 35.6,
 1994: 34.49999999999999,
 1995: 33.5,
 1996: 35.5,
 1997: 30.400000000000002,
 1998: 32.7,
 1999: 38.300000000000004,
 2000: 30.799999999999997,
 2001: 31.9,
 2002: 40.0,
 2003: 33.9,
 2004: 34.5,
 2005: 33.300000000000004,
 2006: 31.099999999999994,
 2007: 33.199999999999

In [11]:
# average the values from 1985-2014, and from 2070-2099
hist_avg = sum([v for k, v in rxnday_results.items() if k >= 1985 and k <= 2014]) / 30
proj_avg = sum([v for k, v in rxnday_results.items() if k >= 2070 and k <= 2099]) / 30

print(f"1985-2014 ensemble average rx{n}day:", hist_avg)
print(f"2070-2099 ensemble average rx{n}day:", proj_avg)

1985-2014 ensemble average rx7day: 33.39
2070-2099 ensemble average rx7day: 40.43666666666666
